## Assignment 2 : Probabilistic Language Model

# Part 1 (15 Marks)
### (1) Display the sentence count
### (2) Build a 4-gram language model.
### (3) Next word prediction - Test it to find the next word using the probability distribution.
• Take any word sequence (trigrams) and find the next word. Display the result of next
word prediction using 5 test sequences
### (4) Sentence generation - Using a random start word to construct a sentence of 10 words.
Generate 5 sentences and publish the result. Avoid using the, in, a or an as the starting
word

#### Install and Import Required Libraries

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from nltk import FreqDist
import matplotlib.pyplot as plt
# import mplcursors
import os
import glob
import numpy as np
import pandas as pd
import regex as re
import pickle
import string
from collections import Counter
import nltk
from nltk import ngrams, FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import pickle
import nltk
from nltk import pos_tag





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#### Mount Google Drive to Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Store All Text Files Inside a List

In [ ]:
corpus_files = glob.glob(os.path.join(r"/content/drive/MyDrive/NLP/Eng_Data/wiki", '*.txt'))

In [ ]:
#forming final preprocessed corpus
# direc = os.listdir('drive/MyDrive/NLP')
final_corpus=[]
for dir in corpus_files:
  f = open(str(dir),'r', encoding = 'utf-8')
  content=f.readlines()
  tem_corpus = '\n'.join(content)
  final_corpus.append(tem_corpus)

In [ ]:
len(final_corpus)

182

In [ ]:
## Sentence Count in 1st file of the whole corpus
sentences = nltk.sent_tokenize(final_corpus[0])
len(sentences)

45241

In [ ]:
Total_Sentence_count=0
for i in range(len(final_corpus)):
  Total_Sentence_count+=len(nltk.sent_tokenize(final_corpus[i]))


### Display the sentence count

In [ ]:
### Total sentences in whole corpus
Total_Sentence_count

7909406

In [ ]:
### Define function to get 4_Gram frequency distribution of the corpus
def get_freq_dist(corpus_text):
  tokenized_sentences = word_tokenize(corpus_text)
  tokenized_sentences= [token.lower() for token in tokenized_sentences]  ### Casefold all the words
  tokenized_sentences=[token for token in tokenized_sentences if token not in string.punctuation and token.isalpha()]  ### Remove Punctuations and Neumaric/Alpha Neumarics
  n = 4
  four_gram_model = list(ngrams([word for word in tokenized_sentences], n))
  fdist = FreqDist(four_gram_model)
  fdist_counter=Counter(fdist)
  return fdist_counter


### Divide whole Corpus of 182 files in 6 batches of size [30,30,30,30,30,32]

#### Get frequency distribution of each batches using get_freq_dist() function

In [ ]:

l=[]
for i in range(150,182):
  print(i)
  l.append(get_freq_dist(final_corpus[i]))


In [ ]:
fdist_counter_182=sum(l, Counter())

### Save the frequency distributions in google drive using pickle.dump()

In [ ]:
with open('/content/drive/MyDrive/NLP/Freq_Dist_182.pkl', 'wb') as fp:
    pickle.dump(fdist_counter_182, fp)

In [ ]:
### Pickeled Frequency Distributions of all 6 batches
pkl_list=['/content/drive/MyDrive/NLP/Freq_Dist_30.pkl','/content/drive/MyDrive/NLP/Freq_Dist_60.pkl',
          '/content/drive/MyDrive/NLP/Freq_Dist_90.pkl','/content/drive/MyDrive/NLP/Freq_Dist_120.pkl',
          '/content/drive/MyDrive/NLP/Freq_Dist_150.pkl','/content/drive/MyDrive/NLP/Freq_Dist_182.pkl']

### Define a function to build a 4-gram language model.

In [ ]:
### Define function to predict the next possible word
def new_word_pred(trigram):
  possible_next_words_counter=Counter()
  for i in pkl_list:
    # print(i)
    file = open(i,'rb')
    freq_dist = pickle.load(file)
    possible_next_words = {item:freq for item, freq in freq_dist.items() if item[:3] == trigram}
    possible_next_words =Counter(possible_next_words)
    possible_next_words_counter=possible_next_words_counter+possible_next_words
    del freq_dist
    del possible_next_words
  if not possible_next_words_counter:
        random_words = word_tokenize(final_corpus[random.choice(range(len(final_corpus)))])
        random_words= [token.lower() for token in random_words]  ### Casefold all the words
        random_words=[token for token in random_words if token not in string.punctuation and token.isalpha()]
        return random.choice(random_words)
  total_count = sum(possible_next_words_counter.values())
  # next_word_probabilities = [(item[3], freq / total_count) for item, freq in possibale_next_words]
  next_word_probabilities = {item[3]: (freq / total_count) for item, freq in possible_next_words_counter.items()}
  # print(len(possible_next_words_counter))
  return (sorted(next_word_probabilities.items(), key=lambda x: x[1])[-1][0])


### Next word prediction - Test it to find the next word using the probability distribution.
• Take any word sequence (trigrams) and find the next word. Display the result of next
word prediction using 5 test sequences

In [ ]:
# Test next word prediction
test_sequences = [("one","of","the"),("end","of","the"),("there","is","a"),("out","of","the"),("where","are","you")]
for seq in test_sequences:
    next_word = new_word_pred(seq)
    print(f"For trigram {seq}, predicted next word: {next_word}")



For trigram ('one', 'of', 'the'), predicted next word: most
For trigram ('end', 'of', 'the'), predicted next word: season
For trigram ('there', 'is', 'a'), predicted next word: small
For trigram ('out', 'of', 'the'), predicted next word: race
For trigram ('where', 'are', 'you'), predicted next word: going


### Define a function to generate sentences using previous next word prediction function

In [ ]:
# Updated generate_sentence function
def generate_sentence(start_word, length=10):
    sentence = [start_word]
    for _ in range(length - 1):
        trigram = tuple(sentence[-3:])
        next_word = new_word_pred(trigram)
        if next_word:
            sentence.append(next_word)
        else:
            break
    # Convert the elements to strings before joining
    return " ".join(str(word) for word in sentence)


### Sentence generation - Using a random start word to construct a sentence of 10 words.
Generate 5 sentences and publish the result. Avoid using the, in, a or an as the starting
word

In [ ]:
# Generate 5 sentences
for _ in range(5):
    random_words = word_tokenize(final_corpus[random.choice(range(len(final_corpus)))])
    random_words= [token.lower() for token in random_words]  ### Casefold all the words
    random_words=[token for token in random_words if token not in string.punctuation and token.isalpha()]
    start_word = random.choice([word for word in random_words if word not in ["the", "in", "a", "an"]])
    sentence = generate_sentence(start_word)
    print(sentence)

church the osteopathic an historical his a a loader as
originally blind is nick in a root advances is with
kabaddi identified powell urchins initially played topics was population stating
tinderbox flowering fortified as the for the first time in
india olympics about event in the united states and canada


In [ ]:
# Generate 5 sentences
for _ in range(5):
    random_words = word_tokenize(final_corpus[random.choice(range(len(final_corpus)))])
    random_words= [token.lower() for token in random_words]  ### Casefold all the words
    random_words=[token for token in random_words if token not in string.punctuation and token.isalpha()]
    start_word = random.choice([word for word in random_words if word not in ["the", "in", "a", "an"]])
    sentence = generate_sentence(start_word)
    print(sentence)


with in panned cloudy bethel played signing american a hemel
run in ontario on the shores of the lake is
example in it was described by the same name by
love an border lead is structure elliott music but switching
minister tribes the is and the community of the basque


# Part 2 (15 Marks)
### (1) Use NLTK to tag all sentences of the corpus using Parts of Speech (POS)
### (2) Recompose the entire corpus using POS instead of original words
• Recompose it sentence by sentence and save them in a file.
• Find the frequency of each structure
• Display the total number of unique structures
### (3) List the most common (5 structures) POS structures/sentence/Grammatical structures.
• This should be done for each sentence in the corpus

Use NLTK to tag all sentences of the corpus using Parts of Speech (POS) and recompose the entire corpus using POS instead of original words and save it in google drive for later use

In [ ]:
structure_counter=Counter()
unique_structure_list=[]
for i in range(len(final_corpus)):
  # Your input corpus
  corpus = final_corpus[i]
  # (1) Tag all sentences of the corpus using Parts of Speech (POS)
  sentences = sent_tokenize(corpus)
  def clean_sentence(sentence):
      words = word_tokenize(sentence)
      cleaned_words = [word for word in words if word.isalpha()]  # Remove numeric/alpha-numeric
      cleaned_sentence = ' '.join(cleaned_words)
      return cleaned_sentence

  tagged_sentences = [pos_tag(word_tokenize(clean_sentence(sentence))) for sentence in sentences]

  # (2) Recompose the entire corpus using POS instead of original words
  # Recompose it sentence by sentence and save them in a file
  with open(r'/content/drive/MyDrive/NLP/recomposed_corpus/'+str(i)+'_POS.txt', 'w') as output_file:
      for tagged_sentence in tagged_sentences:
          recomposed_sentence = ' '.join([tag for word, tag in tagged_sentence])
          output_file.write(recomposed_sentence + '\n')

  # (3) Find the frequency of each structure
  # Display the total number of unique structures
  structures = [' '.join([tag for word, tag in tagged_sentence]) for tagged_sentence in tagged_sentences]

  print("Total number of unique structures in file "+str(i)+" is : "+str(len(Counter(structures))))
  unique_structure_list.append(len(Counter(structures)))

  structure_counter =structure_counter+ Counter(structures)




Total number of unique structures in file 0 is : 39875
Total number of unique structures in file 1 is : 36073
Total number of unique structures in file 2 is : 37442
Total number of unique structures in file 3 is : 39736
Total number of unique structures in file 4 is : 37787
Total number of unique structures in file 5 is : 38129
Total number of unique structures in file 6 is : 24903
Total number of unique structures in file 7 is : 38137
Total number of unique structures in file 8 is : 35951
Total number of unique structures in file 9 is : 35932
Total number of unique structures in file 10 is : 39055
Total number of unique structures in file 11 is : 39063
Total number of unique structures in file 12 is : 37213
Total number of unique structures in file 13 is : 37049
Total number of unique structures in file 14 is : 41146
Total number of unique structures in file 15 is : 36920
Total number of unique structures in file 16 is : 35216
Total number of unique structures in file 17 is : 37055
To

### Total number of unique POS structures in whole corpus

In [ ]:
len(structure_counter)

6162756

In [ ]:
common_structure_counter=sorted(structure_counter.items(), key=lambda x: x[1])[-5:]

 ## List the most common (5 structures) POS structures/sentence/Grammatical structures

In [ ]:
common_structure_counter

[('NNP VBZ DT NN IN NNS IN DT NN NNP', 7726),
 ('NNP NN VBZ DT NNS IN NN IN DT NNP NN', 9363),
 ('NNP NN VBZ DT NN IN DT NNP NN', 9834),
 ('NNP NN VBZ DT NN IN DT NN NNP', 16792),
 ('NNP NN VBZ DT NNS IN NN IN DT NN NNP', 26438)]

In [ ]:
common_structure_keys=[]

for i in common_structure_counter:
  common_structure_keys.append(i[0])

In [ ]:
common_structure_keys

['NNP VBZ DT NN IN NNS IN DT NN NNP',
 'NNP NN VBZ DT NNS IN NN IN DT NNP NN',
 'NNP NN VBZ DT NN IN DT NNP NN',
 'NNP NN VBZ DT NN IN DT NN NNP',
 'NNP NN VBZ DT NNS IN NN IN DT NN NNP']

### Save the structer frequencies of full corpus and most common structures for later use

In [ ]:
import pickle
with open('/content/drive/MyDrive/NLP/structure_counter.pkl', 'wb') as fp:
    pickle.dump(structure_counter, fp)

In [ ]:
import pickle
with open('/content/drive/MyDrive/NLP/common_structure_counter.pkl', 'wb') as fp:
    pickle.dump(common_structure_counter, fp)

##Part 3 (20 Marks)
#(1) Generate a sentence each using the five common structures
• For the same starting word, generate 5 sentences using the five structures. Avoid using
the, in, a or an as the starting word
• Total number of sentences to display - 25 sentences

In [ ]:
import random

# Define the five common POS structures
common_structures = common_structure_keys

# Sample words to start the sentences
starting_words = ["The", "Quick", "Brown", "Fox", "Jumped"]

# Generate 5 sentences for each common structure
sentences = []
for structure in common_structures:
    for word in starting_words:
        if word not in ["The", "In", "A", "An"]:
            sentence = f"{word} {structure}"
            sentences.append(sentence)

# Shuffle the generated sentences
random.shuffle(sentences)

# Display a total of 25 sentences
for i, sentence in enumerate(sentences[:25], 1):
    print(f"Sentence {i}: {sentence}")


Sentence 1: Brown NNP NN VBZ DT NNS IN NN IN DT NNP NN
Sentence 2: Quick NNP NN VBZ DT NN IN DT NNP NN
Sentence 3: Fox NNP VBZ DT NN IN NNS IN DT NN NNP
Sentence 4: Fox NNP NN VBZ DT NNS IN NN IN DT NNP NN
Sentence 5: Fox NNP NN VBZ DT NN IN DT NN NNP
Sentence 6: Jumped NNP VBZ DT NN IN NNS IN DT NN NNP
Sentence 7: Jumped NNP NN VBZ DT NNS IN NN IN DT NN NNP
Sentence 8: Brown NNP NN VBZ DT NN IN DT NN NNP
Sentence 9: Fox NNP NN VBZ DT NNS IN NN IN DT NN NNP
Sentence 10: Quick NNP NN VBZ DT NN IN DT NN NNP
Sentence 11: Brown NNP NN VBZ DT NN IN DT NNP NN
Sentence 12: Quick NNP VBZ DT NN IN NNS IN DT NN NNP
Sentence 13: Jumped NNP NN VBZ DT NN IN DT NNP NN
Sentence 14: Jumped NNP NN VBZ DT NN IN DT NN NNP
Sentence 15: Quick NNP NN VBZ DT NNS IN NN IN DT NNP NN
Sentence 16: Fox NNP NN VBZ DT NN IN DT NNP NN
Sentence 17: Quick NNP NN VBZ DT NNS IN NN IN DT NN NNP
Sentence 18: Brown NNP NN VBZ DT NNS IN NN IN DT NN NNP
Sentence 19: Jumped NNP NN VBZ DT NNS IN NN IN DT NNP NN
Sentence 20: Br

In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
from nltk.corpus import brown

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
# List of starting words (customize as needed)
starting_words = ["apple", "book", "car", "dog", "house"]

# Five common grammatical structures
structures = common_structure_keys
# Words to avoid as starting words
avoid_words = {"the", "in", "a", "an"}



In [ ]:
# Function to generate a sentence for a given structure and starting word
def generate_sentence(structure, starting_word):
    sentence = structure.replace("Noun", starting_word)
    return sentence

In [ ]:
# Generate 5 sentences for each structure with random starting words
sentences = []
for structure in structures:
    for _ in range(5):
        # Choose a random starting word from the list
        starting_word = random.choice(starting_words)
        # Ensure the starting word is not in the list of avoided words
        while starting_word.lower() in avoid_words:
            starting_word = random.choice(starting_words)
        sentence = generate_sentence(structure, starting_word)
        sentence = " ".join([starting_word,sentence])
        sentences.append(sentence)

## Only with pos_tags

In [ ]:
# Display the generated sentences
for i, sentence in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sentence}")


Sentence 1: dog NN NN VBZ DT NNS IN NN IN DT NN NN .
Sentence 2: house NN NN VBZ DT NNS IN NN IN DT NN NN .
Sentence 3: book NN NN VBZ DT NNS IN NN IN DT NN NN .
Sentence 4: house NN NN VBZ DT NNS IN NN IN DT NN NN .
Sentence 5: apple NN NN VBZ DT NNS IN NN IN DT NN NN .
Sentence 6: book NN .
Sentence 7: apple NN .
Sentence 8: dog NN .
Sentence 9: house NN .
Sentence 10: car NN .
Sentence 11: apple NN NN VBZ DT NN IN DT NN NN .
Sentence 12: dog NN NN VBZ DT NN IN DT NN NN .
Sentence 13: house NN NN VBZ DT NN IN DT NN NN .
Sentence 14: apple NN NN VBZ DT NN IN DT NN NN .
Sentence 15: car NN NN VBZ DT NN IN DT NN NN .
Sentence 16: book NN NN VBZ DT NN IN DT JJ NN .
Sentence 17: apple NN NN VBZ DT NN IN DT JJ NN .
Sentence 18: apple NN NN VBZ DT NN IN DT JJ NN .
Sentence 19: book NN NN VBZ DT NN IN DT JJ NN .
Sentence 20: book NN NN VBZ DT NN IN DT JJ NN .
Sentence 21: book NN NN VBZ DT NNS IN NN IN DT JJ NN .
Sentence 22: book NN NN VBZ DT NNS IN NN IN DT JJ NN .
Sentence 23: house NN NN

In [ ]:
from nltk.corpus import treebank

nltk.download('treebank')
nltk.download('maxent_treebank_pos_tagger')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
tag_sentences = treebank.tagged_sents()

# Build a mapping from POS tags to words
pos_to_words = defaultdict(list)
for sentence in tag_sentences:
    for word, pos in sentence:
        pos_to_words[pos].append(word)

In [ ]:
# Define a function to convert POS tags to words
def pos_to_words_fn(pos_sentence):
    words = []
    for pos_tag in pos_sentence:
        word = random.choice(brown.tagged_words(tagset='universal')).word
        words.append(word)
    return " ".join(words)

In [ ]:

# All words in the corpus
all_words = [word for sentence in treebank.sents() for word in sentence]

# Define a POS tagger
pos_tagger = nltk.data.load("taggers/maxent_treebank_pos_tagger/english.pickle")


In [ ]:
# Your 4-gram language model (replace this with your actual ngram_model)
ngram_model = four_gram_model

# Define the structures for sentence generation
structures = common_structure_keys

In [ ]:
def generate_original_sentence(structure, starting_word):
    words = [starting_word]
    current_pos = pos_tagger.tag([starting_word])[0][1]  # Get the POS tag of the starting word
    structure_pos = structure.split()

    for pos in structure_pos[1:]:  # Skip the first POS as it is for the starting word
        if pos in pos_to_words:
            word = random.choice(pos_to_words[pos])
            words.append(word)
        else:
            words.append(pos)
    return ' '.join(words).capitalize()



## Replaced with actual words

In [ ]:
# Generate and display sentences
for _ in range(25):
    starting_word = random.choice(starting_words)
    structure = random.choice(structures)
    sentence = generate_original_sentence(structure, starting_word)
    print(f"Sentence {_+1}: {sentence}")

Sentence 1: Car report is an temperature of the able bottle .
Sentence 2: House list has this sums in globe by a average health .
Sentence 3: House .
Sentence 4: Apple consideration says a sales for % into an separate rate .
Sentence 5: Book charge expects the associate on a facility charge .
Sentence 6: Book .
Sentence 7: Car attempt is a appropriations in addition in a primary factory .
Sentence 8: Apple name has a breaker at the pursuant bill .
Sentence 9: Car volatility gives the struggle into the initial compromise .
Sentence 10: Car country 's the sales for limit of the beautiful home .
Sentence 11: Apple employer is a terms in speculation as a chauffeur % .
Sentence 12: House woman wants the amendment of the christian share .
Sentence 13: Apple school says the week by the local return .
Sentence 14: Book market eliminates a products of fit as the homelessness news .
Sentence 15: Dog .
Sentence 16: Dog year says a institutions in sympathy of the angry evidence .
Sentence 17: Hous

________________________________________________________________________________________________________________________________________________________________